In [ ]:

import numpy as np
import glob
import os
# Path pattern to match all Htriplet subject files
os.chdir('/Users/mariazimmermann/dropbox/encoding_model')
file_paths = sorted(glob.glob('full_model_scores_sub-Htriplet*.npz'))

r_list = []

# Load each subject's 'r' array and append to list
for path in file_paths:
    with np.load(path) as data:
        r_list.append(data['r'])

# Stack and average across subjects
r_array = np.stack(r_list, axis=0) 
print(r_array.shape) # shape = (n_subjects, ...)
r_avg = np.mean(r_array, axis=0)    # shape = (...)
print(r_avg.shape)
np.save('full_model_hearing.npy', r_avg)
# r_avg now contains the average map across all subjects

In [ ]:
import numpy as np
import glob
import os

# Set working directory
os.chdir('/Users/mariazimmermann/Dropbox/encoding_model')

# Load ISC mask
isc_mask = np.load('ISC_mask.npy')  # shape = (n_voxels,) or (x,y,z), assume already boolean or 0/1

# Load full model scores for each subject
file_paths = sorted(glob.glob('full_model_scores_sub-Htriplet*.npz'))
r_list = []

for path in file_paths:
    with np.load(path) as data:
        r_list.append(data['r'])  # shape = (n_voxels,)

# Stack: shape (n_subjects, n_voxels)
r_array = np.stack(r_list, axis=0)

# Apply ISC mask: keep only voxels where ISC_mask > 0
if isc_mask.shape != r_array.shape[1:]:
    raise ValueError("Shape mismatch between ISC mask and r data")
r_array_masked = r_array[:, isc_mask > 0]  # shape = (n_subjects, n_masked_voxels)

# Compute true mean
true_mean = np.mean(r_array_masked, axis=0)  # shape = (n_masked_voxels,)

# Run permutation test by sign-flipping (nonparametric paired test)
n_permutations = 10000
n_subjects = r_array_masked.shape[0]
perm_means = np.zeros((n_permutations, r_array_masked.shape[1]))

for i in range(n_permutations):
    signs = np.random.choice([1, -1], size=n_subjects)[:, None]  # shape (n_subjects, 1)
    permuted = r_array_masked * signs
    perm_means[i] = np.mean(permuted, axis=0)

# Compute one-sided p-values: proportion of permuted means >= true mean
p_values = np.mean(perm_means >= true_mean[None, :], axis=0)

# Optional: FDR correction
from statsmodels.stats.multitest import fdrcorrection
_, p_fdr = fdrcorrection(p_values)

# Save outputs
np.save('full_model_hearing_masked.npy', true_mean)
np.save('full_model_hearing_masked_pvals.npy', p_values)
np.save('full_model_hearing_masked_pvals_fdr.npy', p_fdr)

print("Permutation test complete.")
print(f"True mean shape: {true_mean.shape}, p-value shape: {p_values.shape}")


In [ ]:

import numpy as np
import glob
import os
# Path pattern to match all Htriplet subject files
os.chdir('/Users/mariazimmermann/dropbox/encoding_model')
file_paths = sorted(glob.glob('full_model_scores_sub-Htriplet*.npz'))

r_list = []

# Load each subject's 'r' array and append to list
for path in file_paths:
    with np.load(path) as data:
        r_list.append(data['r'])

# Stack and average across subjects
r_array = np.stack(r_list, axis=0) 
print(r_array.shape) # shape = (n_subjects, ...)
r_avg = np.mean(r_array, axis=0)    # shape = (...)
print(r_avg.shape)
np.save('full_model_hearing.npy', r_avg)
# r_avg now contains the average map across all subjects

In [ ]:
import numpy as np
import glob
import os

# Change to the appropriate directory
os.chdir('/Users/mariazimmermann/Dropbox/encoding_model')

# Find all relevant .npz files
file_paths = sorted(glob.glob('split_r_sub-Htriplet*.npz'))

# Initialize lists for each feature
arousal_list = []
valence_list = []
tom_list = []
soc_list = []
no_list = []
hsv_list = []
motion_list = []
faces_list = []

# Load each subject's data
for path in file_paths:
    with np.load(path) as data:
        arousal_list.append(data['AROUSAL'])
        valence_list.append(data['VALENCE'])
        tom_list.append(data['TOM'])
        soc_list.append(data['SOC'])
        no_list.append(data['NO'])
        hsv_list.append(data['HSV'])
        motion_list.append(data['MOTION_PCA'])
        faces_list.append(data['FACES'])

# Define a helper to average and save
def average_and_save(data_list, name):
    arr = np.stack(data_list, axis=0)  # shape: (n_subjects, ...)
    avg = np.mean(arr, axis=0)
    np.save(f'{name}_hearing.npy', avg)
    print(f'{name} shape: {avg.shape}')

# Average and save each feature
average_and_save(arousal_list, 'arousal')
average_and_save(valence_list, 'valence')
average_and_save(tom_list, 'tom')
average_and_save(soc_list, 'soc')
average_and_save(no_list, 'no')
average_and_save(hsv_list, 'hsv')
average_and_save(motion_list, 'motion')
average_and_save(faces_list, 'faces')


In [ ]:
import numpy as np
import glob
import os

# Change to the appropriate directory
os.chdir('/Users/mariazimmermann/Dropbox/encoding_model')

# Find all relevant .npz files
file_paths = sorted(glob.glob('split_r_sub-Dtriplet*.npz'))

# Initialize lists for each feature
arousal_list = []
valence_list = []
tom_list = []
soc_list = []
no_list = []
hsv_list = []
motion_list = []
faces_list = []

# Load each subject's data
for path in file_paths:
    with np.load(path) as data:
        arousal_list.append(data['AROUSAL'])
        valence_list.append(data['VALENCE'])
        tom_list.append(data['TOM'])
        soc_list.append(data['SOC'])
        no_list.append(data['NO'])
        hsv_list.append(data['HSV'])
        motion_list.append(data['MOTION_PCA'])
        faces_list.append(data['FACES'])

# Define a helper to average and save
def average_and_save(data_list, name):
    arr = np.stack(data_list, axis=0)  # shape: (n_subjects, ...)
    avg = np.mean(arr, axis=0)
    np.save(f'{name}_deaf.npy', avg)
    print(f'{name} shape: {avg.shape}')

# Average and save each feature
average_and_save(arousal_list, 'arousal')
average_and_save(valence_list, 'valence')
average_and_save(tom_list, 'tom')
average_and_save(soc_list, 'soc')
average_and_save(no_list, 'no')
average_and_save(hsv_list, 'hsv')
average_and_save(motion_list, 'motion')
average_and_save(faces_list, 'faces')